In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 85 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595737 sha256=e7b7554e33a7a6572b375c6af305fcdfb3945bab394ac23a0cabf3eb745c962e
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 769 kB 5.3 MB/s 
     |████████████████████████████████| 3.0 MB 30.4 MB/s 
     |████████████████████████████████| 895 kB 45.1 MB/s 


In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-krdtoti2
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-krdtoti2
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=d1cd9b7604a4113f07578a0d45f38b2b1c1364442f521c8cc9bbd2cf9c78198e
  Stored in directory: /tmp/pip-ephem-wheel-cache-h9i3thk0/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")
#device = torch.device("cpu")


#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
import pandas as pd
chatbot_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/감성대화.xlsx')
chatbot_data_new = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/한국어_단발성_대화_데이터셋.xlsx')

In [ ]:
chatbot_data = chatbot_data.loc[(chatbot_data['감정_대분류'] != "당황")]
len(chatbot_data)

8373

In [ ]:
chatbot_data.sample(n=3)


,연령,성별,상황키워드,감정_대분류,감정_소분류,사람문장1,기계문장1,사람문장2,기계문장2,사람문장3,기계문장3
6857,중년,여성,건강,슬픔,실망한,건강만큼은 자신했는데 요즘 자주 어지러워.,자주 어지러워서 걱정되시는군요.,살을 빼고 싶어서 단식했거든. 너무 살이 쪄서 거울 보기가 싫어.,단식으로 어지러움이 생기셨군요.,난 의지가 약해서 소식이 안 돼. 맛있으면 폭식을 하거든. 아예 안 먹는 게 나은 ...,건강하게 살을 빼는 방법이 있을까요?
1392,청소년,남성,가족관계,슬픔,염세적인,가족과 함께 시간을 보내는 데에는 관심도 재미도 없어.,가족과 시간을 보내는 것이 재미 없으시군요.,친구들처럼 나도 가족들과 화목한 시간을 보내고 싶어.,가족들과 화목한 시간을 보내려면 어떻게 해야 좋을까요?,가족들과 함께하는 해외여행을 한번 제안해볼 거야.,가족들과 해외여행을 가서 화목한 시간을 보내길 바라요.
9386,노년,남성,"건강, 죽음",불안,초조한,내가 죽으면 내 장례식장에 와서 가족들은 슬퍼하겠지?,내 장례식장에서 가족들이 슬퍼할까 봐 걱정이 크군요.,응. 남겨질 가족들을 생각하니 너무 불안해.,어떻게 하면 상황이 나이질 수 있을까요?,지금이라도 내 건강 관리에 더 힘을 써야겠어.,건강에 더 신경을 쓸 생각이시군요.


In [ ]:
chatbot_data.loc[(chatbot_data['감정_대분류'] == "불안"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['감정_대분류'] == "상처"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['감정_대분류'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['감정_대분류'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
#chatbot_data.loc[(chatbot_data['감정_대분류'] == "당황"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['감정_대분류'] == "기쁨"), 'Emotion'] = 4  #행복 => 5

In [ ]:
data_list = []
for q, label in zip(chatbot_data['사람문장1'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(int(label)))

    data_list.append(data)


In [ ]:
print(data_list[0])
print(data_list[600])
print(data_list[1200])
print(data_list[1800])
print(data_list[2400])
print(data_list[3000])
print(data_list[-1])

['오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.', '4']
['나를 계속 괴롭혔던 애가 너무 미워서 오늘은 그 애 가방 끈을 자르고 도망쳤어.', '2']
['엄마가 나한테 하는 말이 얼마나 큰 상처가 되는지 마음이 너무 아프다.', '3']
['날 따돌리는 친구가 내가 돈이 없어서 수학여행에 가지 못하는 것을 소문 냈어.', '1']
['부모님 덕분에 드디어 신혼 집을 구했어. 그것도 우리 형편에 비해 좋은 집으로.', '4']
['몇 달째 실적이 나지 않아 고민이야.', '0']
['아들이 갑자기 찾아왔는데 손자가 희귀병이래. 너무 두려워.', '0']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

6279
2094


In [ ]:
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 60
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
chatbot_data['Emotion']

0       4.0
1       4.0
2       4.0
3       4.0
4       4.0
       ... 
9994    3.0
9995    3.0
9996    1.0
9997    0.0
9998    0.0
Name: Emotion, Length: 8373, dtype: float64

In [ ]:
data_train[0]

(array([   2, 3501, 3257, 3114, 5760, 1435, 1840, 7101, 6553, 5025, 7799,
        1965, 4213, 6604, 6855,  517,   54, 1185, 3224, 1958, 6371, 4524,
        4634, 7096, 6844,  517,   54,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1], dtype=int32),
 array(28, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 4)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/99 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.8184177875518799 train acc 0.171875
epoch 1 train acc 0.4323818542568542


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 2 test acc 0.6697751976284585


  0%|          | 0/99 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.595665693283081 train acc 0.84375
epoch 3 train acc 0.7477678571428572


  0%|          | 0/33 [00:00<?, ?it/s]

epoch 3 test acc 0.6746953227931489


In [ ]:
x_test = []
for i in dataset_test:
  x_test.append(int(i[1]))

x_test
y_test

[4,
 4,
 2,
 0,
 4,
 4,
 2,
 2,
 3,
 0,
 2,
 4,
 3,
 1,
 1,
 4,
 2,
 2,
 4,
 3,
 3,
 3,
 4,
 2,
 4,
 3,
 4,
 0,
 1,
 0,
 4,
 2,
 3,
 0,
 2,
 0,
 0,
 2,
 2,
 1,
 2,
 1,
 0,
 3,
 4,
 3,
 1,
 3,
 0,
 0,
 1,
 4,
 3,
 4,
 4,
 0,
 3,
 2,
 3,
 0,
 1,
 2,
 0,
 4,
 2,
 3,
 1,
 4,
 1,
 2,
 2,
 0,
 3,
 2,
 1,
 4,
 1,
 4,
 0,
 4,
 1,
 2,
 2,
 2,
 0,
 4,
 4,
 1,
 2,
 1,
 1,
 1,
 0,
 3,
 1,
 4,
 4,
 3,
 2,
 1,
 4,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 3,
 2,
 0,
 4,
 4,
 3,
 0,
 1,
 0,
 4,
 1,
 2,
 0,
 0,
 3,
 4,
 1,
 4,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 2,
 3,
 0,
 4,
 0,
 1,
 2,
 2,
 1,
 3,
 2,
 3,
 4,
 1,
 1,
 0,
 1,
 4,
 0,
 3,
 1,
 2,
 1,
 4,
 3,
 0,
 1,
 4,
 1,
 4,
 2,
 2,
 1,
 2,
 1,
 2,
 4,
 4,
 3,
 1,
 3,
 2,
 3,
 0,
 3,
 3,
 3,
 1,
 0,
 1,
 0,
 1,
 4,
 1,
 0,
 2,
 1,
 2,
 0,
 3,
 3,
 4,
 1,
 3,
 2,
 3,
 4,
 0,
 3,
 0,
 1,
 0,
 2,
 2,
 2,
 3,
 2,
 2,
 4,
 2,
 3,
 3,
 1,
 1,
 3,
 2,
 1,
 2,
 0,
 3,
 4,
 4,
 3,
 4,
 1,
 2,
 3,
 2,
 3,
 4,
 3,
 0,
 1,
 0,
 0,
 3,
 3,
 3,
 4,
 1,
 4,
 0,
 1,
 0,
 4,
 3,
 1,


In [ ]:
# 데이터셋 각 라벨에 대한 정확도 측정
#y_predicted = bilstm_model.predict([X_test])
#pred_tags = f1score.sequences_to_tags(y_predicted)
#test_tags = f1score.sequences_to_tags(y_test)

#print(classification_report(test_tags, pred_tags))

from sklearn.metrics import confusion_matrix, classification_report

df = pd.DataFrame(columns=['Predicted Labels', 'Actual Labels'])
df['Predicted Labels'] = x_test
df['Actual Labels'] = y_test

df.head(10)

print(classification_report(x_test, y_test,target_names=['불안','상처','분노','슬픔','기쁨']))

              precision    recall  f1-score   support

          불안       0.62      0.73      0.67       397
          상처       0.60      0.57      0.58       449
          분노       0.66      0.64      0.65       393
          슬픔       0.61      0.56      0.58       433
          기쁨       0.89      0.89      0.89       422

    accuracy                           0.68      2094
   macro avg       0.68      0.68      0.68      2094
weighted avg       0.68      0.68      0.67      2094



In [ ]:
y_test = []
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = data_test
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            #print(logits)
            if np.argmax(logits) == 0:
                test_eval.append("불안이")
            elif np.argmax(logits) == 1:
                test_eval.append("상처가")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("행복이")
            elif np.argmax(logits) == 5:
                test_eval.append("당황이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")
            
            y_test.append(np.argmax(logits))
        #print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")


In [ ]:
sentence = input("하고싶은 말을 입력해주세요 : ")
predict(sentence)
print("\n")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
